### Deploy registered AG model in MLFlow for RT Inference

In [6]:
! pip install fastavro

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 74.3 MB/s eta 0:00:00


In [55]:
import boto3
import fastavro
import matplotlib
import mlflow
import mlflow.sagemaker as mfs
from mlflow import MlflowClient
from sagemaker.serve import SchemaBuilder, ModelBuilder, Mode
import sagemaker

import json
import pandas as pd
import mlflow.pyfunc
import tempfile
import os 
import io
import boto3
import time
from fastavro import reader as avro_reader
from sagemaker.serve import ModelBuilder, Mode, SchemaBuilder


In [66]:
bucket = 'ag-example-timeseries'
avro_prefix = 'avro-inf-stream'

# Create S3 client
s3 = boto3.client("s3")


mlflow_uri = "arn:aws:sagemaker:us-east-1:543531862107:mlflow-tracking-server/ag-ex-timeseries"  # with sagemaker-mlflow plugin
mlflow_experiment   = "autogluon-timeseries"
region      = sagemaker.Session().boto_region_name
session     = sagemaker.Session()
role        = sagemaker.get_execution_role() 

### Generate dummy data

In [10]:
import io, json, time, uuid, threading, datetime as dt
import numpy as np
import pandas as pd
import boto3
from fastavro import writer, parse_schema

In [11]:
ITEM_IDS        = ["A", "B"]                        # a couple of series to demo
FREQ_SECS       = 5                                 # new file cadence
HORIZON         = 24  

In [13]:
AVRO_SCHEMA = {
    "type": "record",
    "name": "TimePoint",
    "fields": [
        {"name": "item_id",        "type": "string"},
        {"name": "timestamp",      "type": "string"},  # ISO 8601
        {"name": "target",         "type": ["null", "double"], "default": None},
        {"name": "random_feature", "type": ["null", "double"], "default": None},
    ],
}
PARSED_SCHEMA = parse_schema(AVRO_SCHEMA)

_stop_stream = {"flag": False}

def _sine(i, base=10.0, noise=0.3):
    return base + 2.0*np.sin(i/6.0) + np.random.randn()*noise

def write_dummy_avro_loop():
    i = 0
    while not _stop_stream["flag"]:
        now = pd.Timestamp.utcnow().floor("s")
        rows = []
        for item in ITEM_IDS:
            rows.append({
                "item_id": item,
                "timestamp": (now).isoformat(),
                "target": float(_sine(i + hash(item)%7)),      # last observed target (optional at inference)
                "random_feature": float(np.random.randn()),    # example past/known covariate
            })
        buf = io.BytesIO()
        writer(buf, PARSED_SCHEMA, rows)
        buf.seek(0)
        key = f"{avro_prefix}{now.strftime('%Y/%m/%d/%H%M%S')}_{uuid.uuid4().hex[:8]}.avro"
        s3.upload_fileobj(buf, bucket, key)
        print(f"[gen] wrote {len(rows)} rows to s3://{bucket}/{key}")
        i += 1
        time.sleep(FREQ_SECS)

# start writer thread
t = threading.Thread(target=write_dummy_avro_loop, daemon=True)
t.start()



[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220031_fd1c6ddd.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220036_ad5dac74.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220041_bd4b67bd.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220046_ea79e7ed.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220051_4906e4b8.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220056_b5381fe1.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220102_b0facca5.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220107_1c6d9ef8.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220112_7781ba66.avro
[gen] wrote 2 rows to s3://ag-example-timeseries/avro-inf-stream2025/09/04/220117_fe0234f7.avro
[gen] wrote 2 rows to s3://ag-example-ti

In [25]:
# Stop it later with:
_stop_stream["flag"] = True
t.join()

### Inference 

In [78]:
mlflow.set_tracking_uri(mlflow_uri)

# Define the model URI using the registry syntax
model_name = "ag-ex-model"  # Use the name defined in the helper_functions.py
model_uri = f"models:/{model_name}/latest"

from sagemaker import image_uris

# Get the GPU image URI for PyTorch inference
gpu_image_uri = image_uris.retrieve(
    framework="pytorch",
    region=sagemaker.Session().boto_region_name,
    version="2.2.0", # Match your training framework version
    py_version="py310", # Match your Python version
    instance_type="ml.g4dn.2xlarge", # Specify a GPU instance type
    image_scope="inference",
)

print(f"GPU Image URI: {gpu_image_uri}")

GPU Image URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.2.0-gpu-py310


In [ ]:
# Define IO
# NOTE: This schema is for training/model logging only. 
# Your live inference will use raw Avro bytes.
sample_input = pd.DataFrame({
    "item_id": ["A", "A", "A", "B", "B"],
    "timestamp": pd.to_datetime([
        "2025-01-01 00:00:00",
        "2025-01-01 01:00:00",
        "2025-01-01 02:00:00",
        "2025-01-01 00:00:00",
        "2025-01-01 01:00:00",
    ]),
    "target": [10.0, 11.2, 12.1, 8.7, 9.1],
    "random_feature": [0.2, -0.1, 0.3, -0.4, 0.0],
})
sample_output = pd.DataFrame({
    "item_id": ["A", "A", "B", "B"],
    "timestamp": pd.to_datetime([
        "2025-01-01 03:00:00", "2025-01-01 04:00:00",
        "2025-01-01 02:00:00", "2025-01-01 03:00:00",
    ]),
    "mean": [12.6, 13.0, 9.4, 9.7],
})

ts_schema_builder = SchemaBuilder(
    sample_input=sample_input,
    sample_output=sample_output,
)

# Create model builder with the schema builder.
# Replace with your S3 path
source_path = "s3://ag-example-timeseries/mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/"
ENDPOINT_NAME = "ag-ex-endpoint"
avro_prefix = f"s3://{bucket}/" 
FREQ_SECS = 5

model_metadata = {
    "MLFLOW_MODEL_PATH": source_path,
    "SAGEMAKER_DEFAULT_ENDPOINT_NAME": ENDPOINT_NAME,
    # This metadata is specific to Sagemaker's internal handling, not our custom wrapper
}

print(source_path)

model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=ts_schema_builder,
    image_uri=gpu_image_uri,
    role_arn=role,
    model_metadata=model_metadata,
)

# Build & deploy endpoint
built_model = model_builder.build()
predictor = built_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.2xlarge",
)

print("Deployed SageMaker endpoint:", predictor.endpoint_name)

# -------------------------------------------------------------------
# Live inference polling logic
# -------------------------------------------------------------------
smr = boto3.client("sagemaker-runtime", region_name=REGION)

def list_new_objects(bucket: str, prefix: str, seen: set[str]) -> list[str]:
    keys = []
    token = None
    while True:
        kw = dict(Bucket=bucket, Prefix=prefix, MaxKeys=1000)
        if token: kw["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kw)
        for obj in resp.get("Contents", []):
            k = obj["Key"]
            if k.endswith(".avro") and k not in seen:
                keys.append(k)
        token = resp.get("NextContinuationToken")
        if not token:
            break
    return sorted(keys)

def invoke_endpoint_avro(payload: bytes) -> pd.DataFrame:
    # Set ContentType to 'avro/binary' for the endpoint to handle
    resp = smr.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="avro/binary",
        Accept="application/json",
        Body=payload,
    )
    # The response is JSON; read to pandas
    out = pd.read_json(io.BytesIO(resp["Body"].read()))
    # Ensure timestamp is datetime if present
    if "timestamp" in out.columns:
        out["timestamp"] = pd.to_datetime(out["timestamp"])
    return out

def read_avro_s3_to_bytes(bucket: str, key: str) -> bytes:
    bio = io.BytesIO()
    s3.download_fileobj(bucket, key, bio)
    bio.seek(0)
    return bio.getvalue()

def live_infer_and_plot(loop_seconds=300, poll_every=5):
    """
    Polls for new Avro files every `poll_every` seconds for up to `loop_seconds`,
    invokes the endpoint with all new rows, and live-updates a simple line chart.
    """
    seen = set()
    all_preds = []

    start = time.time()
    while time.time() - start < loop_seconds:
        new_keys = list_new_objects(bucket, avro_prefix, seen)
        if new_keys:
            for k in new_keys:
                try:
                    payload = read_avro_s3_to_bytes(bucket, k)
                    preds = invoke_endpoint_avro(payload)
                    all_preds.append(preds)
                except Exception as e:
                    print(f"[invoke] {k} failed: {e}")
                finally:
                    seen.add(k)

        # Plot
        if all_preds:
            # ... (plotting logic is the same) ...
            pred_df = pd.concat(all_preds, ignore_index=True)
            y_col = "mean"
            if y_col not in pred_df.columns:
                qcols = [c for c in pred_df.columns if c.startswith("0.") or c.startswith("quantile")]
                y_col = qcols[0] if qcols else None
            
            clear_output(wait=True)
            plt.figure(figsize=(9, 4))
            if y_col and {"item_id","timestamp", y_col}.issubset(pred_df.columns):
                for item, g in pred_df.sort_values("timestamp").groupby("item_id"):
                    plt.plot(g["timestamp"], g[y_col], label=str(item))
                plt.title(f"Live forecasts (last {int(time.time()-start)}s)")
                plt.xlabel("timestamp")
                plt.ylabel(y_col if y_col else "prediction")
                plt.legend()
                plt.tight_layout()
                plt.show()
            else:
                print("[plot] Prediction DataFrame does not have the expected columns:",
                      pred_df.columns.tolist())
        else:
            print("[plot] Waiting for first predictions...")

        time.sleep(poll_every)

# Example invocation
# live_infer_and_plot(loop_seconds=180, poll_every=FREQ_SECS)


ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
ModelBuilder: INFO:     Downloading mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/MLmodel to /tmp/sagemaker/model-builder/17e90be689dc11f0aaeea2e5a28b0157/MLmodel


s3://ag-example-timeseries/mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/


ModelBuilder: INFO:     Downloading mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/conda.yaml to /tmp/sagemaker/model-builder/17e90be689dc11f0aaeea2e5a28b0157/conda.yaml
ModelBuilder: INFO:     Downloading mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/input_example.json to /tmp/sagemaker/model-builder/17e90be689dc11f0aaeea2e5a28b0157/input_example.json
ModelBuilder: INFO:     Downloading mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/python_env.yaml to /tmp/sagemaker/model-builder/17e90be689dc11f0aaeea2e5a28b0157/python_env.yaml
ModelBuilder: INFO:     Downloading mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/python_model.pkl to /tmp/sagemaker/model-builder/17e90be689dc11f0aaeea2e5a28b0157/python_model.pkl
ModelBuilder: INFO:     Downloading mlflow/1/models/m-b95d1216a1ef4db1852c664a05470f30/artifacts/requirements.txt to /tmp/sagemaker/model-builder/17e90be689dc11f0aaeea2e5a28b0157/requirements.txt
ModelBuilder: INFO:     Downlo

[09/05/25 21:21:28] INFO     Skipping auto detection as the image uri is provided              ]8;id=11468;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/model_builder.py\model_builder.py]8;;\:]8;id=267052;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/model_builder.py#357\357]8;;\
                             763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.                     
                             2.0-gpu-py310                                                                         

INFO:sagemaker_core.main.utils:Skipping auto detection as the image uri is provided 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.2.0-gpu-py310
ModelBuilder: DEBUG:     Packaging provided requirements.txt from /tmp/sagemaker/model-builder/17e90be689dc11f0aaeea2e5a28b0157/requirements.txt
ModelBuilder: DEBUG:     Uploading the model resources to bucket=sagemaker-us-east-1-543531862107, key_prefix=pytorch-inference-2025-09-05-21-21-28-830.
Uploading model artifacts: 100%|███████████████████████| 20503/20503 [00:00<00:00, 149636.01bytes/s]
ModelBuilder: DEBUG:     Model resources uploaded to: s3://sagemaker-us-east-1-543531862107/pytorch-inference-2025-09-05-21-21-28-830/serve.tar.gz
ModelBuilder: DEBUG:     ModelBuilder metrics emitted.
ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defa

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:55                                                                                   │
│                                                                                                  │
│    52                                                                                            │
│    53 # Build & deploy endpoint                                                                  │
│    54 built_model = model_builder.build()                                                        │
│ ❱  55 predictor = built_model.deploy(                                                            │
│    56 │   initial_instance_count=1,                                                              │
│    57 │   instance_type="ml.g4dn.2xlarge",                                                       │
│    58 )                                                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/utils/telemetry_logger.py:184 in wrapper │
│                                                                                                  │
│   181 │   │   │   │   )                                                                          │
│   182 │   │   │                                                                                  │
│   183 │   │   │   if caught_ex:                                                                  │
│ ❱ 184 │   │   │   │   raise caught_ex                                                            │
│   185 │   │   │                                                                                  │
│   186 │   │   │   return response                                                                │
│   187                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/utils/telemetry_logger.py:104 in wrapper │
│                                                                                                  │
│   101 │   │   │                                                                                  │
│   102 │   │   │   start_timer = perf_counter()                                                   │
│   103 │   │   │   try:                                                                           │
│ ❱ 104 │   │   │   │   response = func(self, *args, **kwargs)                                     │
│   105 │   │   │   except (                                                                       │
│   106 │   │   │   │   ModelBuilderException,                                                     │
│   107 │   │   │   │   exceptions.CapacityError,                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/serve/builder/model_builder.py:620 in          │
│ _model_builder_deploy_wrapper                                                                    │
│                                                                                                  │
│    617 │   │   │   kwargs["endpoint_logging"] = True                                             │
│    618 │   │   kwargs.pop("mode", None)                                                          │
│    619 │   │   self.pysdk_model.role = kwargs.pop("role", self.pysdk_model.role)                 │
│ ❱  620 │   │   predictor = self._original_deploy(                                                │
│    621 │   │   │   *args,                                                                        │
│    622 │   │   │   instance_type=instance_type,            